# TEMP_loom_load

In [10]:
# Libraries

# For loading the loom file.
import anndata
import loompy
import scanpy

In [11]:
pbmc = anndata.io.read_loom(
    "/media/apollo/Samsung_T5/transfer/mayur/loom/BMP-1ST2.loom"
)

/home/apollo/mosaic/envs/3d_rna_velocity/lib/python3.11/site-packages/anndata/_core/anndata.py:1758: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


In [30]:
pbmc

AnnData object with n_obs × n_vars = 13234 × 61860
    var: 'Accession', 'Chromosome', 'End', 'Start', 'Strand'
    layers: 'matrix', 'ambiguous', 'spliced', 'unspliced'

# Metric

In [61]:
# Make sure the AnnData object consists of disjoint sets of genes.

# Spliced genes.
spliced = []

# Row and column indices.
nonzeros = pbmc.layers['spliced'].nonzero()

row_indices = list(nonzeros[0])
column_indices = list(nonzeros[1])

for i in range(len(row_indices)):
    spliced.append((row_indices[i], column_indices[i]))

# Unspliced genes.
unspliced = []

# Row and column indices.
nonzeros = pbmc.layers['unspliced'].nonzero()

row_indices = list(nonzeros[0])
column_indices = list(nonzeros[1])
#print(row_indices)

for i in range(len(row_indices)):
    unspliced.append((row_indices[i], column_indices[i]))

# Ambiguous genes.
ambiguous = []

# Row and column indices.
nonzeros = pbmc.layers['ambiguous'].nonzero()

row_indices = list(nonzeros[0])
column_indices = list(nonzeros[1])
#print(row_indices)

for i in range(len(row_indices)):
    ambiguous.append((row_indices[i], column_indices[i]))

Check for unique row and column combinations.

In [68]:
print(len(set(spliced)) == len(spliced))
print(len(set(unspliced)) == len(unspliced))
print(len(set(ambiguous)) == len(ambiguous))

True
True
True


Make sure no overlap between the different categories.

In [79]:
# Overlaps?
spliced_unspliced = set.intersection(set(spliced), set(unspliced))
spliced_ambgiuous = set.intersection(set(spliced), set(ambiguous))
unspliced_ambiguous = set.intersection(set(unspliced), set(ambiguous))

print(len(spliced_unspliced))
print(len(spliced_ambgiuous))
print(len(unspliced_ambiguous))
print(len(set.intersection(spliced_unspliced, unspliced_ambiguous)))

3517
2139
1371
464


# METRIC

### We can consider how related the ambiguous reads are to the gene length/number of isoforms.  If we see a reasonable trend relating the gene length to the ambiguity, then we may have confounding (paper source...)

In [ ]:
# Which genes are ambiguous?

In [36]:
pbmc.layers['ambiguous'].nonzero()

(array([    0,     8,     8, ..., 13223, 13223, 13228], dtype=int32),
 array([24006,  8748,  9016, ...,  8748, 19828, 45474], dtype=int32))

In [26]:
pbmc.var

,Accession,Chromosome,End,Start,Strand
Gene,,,,,
WASH7P,ENSG00000227232,1,29570,14404,-
MIR6859-1,ENSG00000278267,1,17436,17369,-
FAM138A,ENSG00000237613,1,36081,34554,-
ENSG00000238009,ENSG00000238009,1,133723,89295,-
ENSG00000239945,ENSG00000239945,1,91105,89551,-
...,...,...,...,...,...
ANKRD36P1,ENSG00000188399,Y,26420535,26409815,+
ENSG00000277146,ENSG00000277146,Y,26454092,26453982,+
TPTE2P4,ENSG00000215506,Y,26579690,26508213,+
